#### URLs
* Get all Categories of Shopee Mall: https://shopee.vn/api/v4/official_shop/get_categories?tab_type=0
* Get list of all Shop in each category: https://shopee.vn/api/v4/official_shop/get_shops_by_category?need_zhuyin=0&category_id=11035954
* Get shop details: https://shopee.vn/api/v4/shop/get_shop_detail?username=batianda.vn
* Get Hot deals of each shop: https://shopee.vn/api/v4/search/search_items?by=relevancy&limit=6&match_id=97169520&newest=0&order=desc&page_type=shop&scenario=PAGE_OTHERS&shop_categoryids=13938834&version=2
* Get list of products of each shop: https://shopee.vn/api/v4/search/search_items?by=pop&limit=30&match_id=97169520&newest=0&order=desc&page_type=shop&scenario=PAGE_OTHERS&version=2 

In [129]:
import urllib,json
import pandas as pd
import numpy as np
import time,datetime
from sqlalchemy import create_engine
import pymysql
import pyodbc

In [147]:
mall_url = 'https://shopee.vn/api/v4/official_shop/get_categories?tab_type=0'

mall_response = urllib.request.urlopen(mall_url)
mall_data = json.loads(mall_response.read())['data']['categories']

df_m = pd.DataFrame(mall_data)
df_m.to_csv('MallList.csv',encoding='utf8-bom',index=False)
#transfer_data_to_db(mall_data['data']['categories'],'malls_list')

LookupError: unknown encoding: utf8-bom

In [140]:
def encodeData(list):
    result=[]
    for l in list:
        result.append(l.encode('utf-8'))
    return result

In [5]:
cat_list = [11035954,11036793,11036670]
cat_url = 'https://shopee.vn/api/v4/official_shop/get_shops_by_category?category_id=11035954'

cat_response = urllib.request.urlopen(cat_url)
cat_data = json.loads(cat_response.read())
cat_data

{'error': 0,
 'error_msg': None,
 'data': {'brands': [{'index': 'A',
    'total': 10,
    'brand_ids': [{'username': 'a4t_official_store',
      'brand_name': 'A4 Tech',
      'shopid': 269477577,
      'logo': '98fb985087494766588272f81d38cb1f',
      'logo_pc': '3f2e7bd409c2cd7a385834990ce430e4',
      'shop_collection_id': 0,
      'ctime': 1620981896,
      'brand_label': 0},
     {'username': 'acer_flagshipstore',
      'brand_name': 'Acer Flagship Store',
      'shopid': 333430029,
      'logo': '3c828a89fcd3f8d4649b5d661e2f4ab5',
      'logo_pc': '5ed287b3a5f2076ba13f46aee0b0cf2e',
      'shop_collection_id': 0,
      'ctime': 1620981896,
      'brand_label': 0},
     {'username': 'acer_flagship_hch_store',
      'brand_name': 'Acer Hàng Chính Hiệu',
      'shopid': 106066556,
      'logo': '3c828a89fcd3f8d4649b5d661e2f4ab5',
      'logo_pc': '5ed287b3a5f2076ba13f46aee0b0cf2e',
      'shop_collection_id': 0,
      'ctime': 1620981896,
      'brand_label': 0},
     {'username': '

In [127]:
def get_brands_list(cat_data, category_id):
    brands_list = []
    for brands in cat_data['data']['brands']:
        for b in brands['brand_ids']:
            temp = {'username':b['username'],'brand_name':b['brand_name'],'shopid':b['shopid'],'category_id':category_id}
            brands_list.append(temp)
    return brands_list

In [124]:
def get_shop_detail_list(brands_list):

    shop_detail_list = []
    shop_url  = 'https://shopee.vn/api/v4/shop/get_shop_detail?username='

    for brand in brands_list:
        temp_url = shop_url + brand['username']
        shop_response = urllib.request.urlopen(temp_url)
        b = json.loads(shop_response.read())
        temp = {'shopid':b['data']['shopid'],
                'shopname':b['data']['name'].title(),
                'username':b['data']['account']['username'],
                'userid':b['data']['userid'],
                'country':b['data']['country'],
                'description':b['data']['description'],
                'rating_star':b['data']['rating_star'],
                'rating_normal':b['data']['rating_normal'],
                'rating_bad':b['data']['rating_bad'],
                'rating_good':b['data']['rating_good'],
                'shop_location':b['data']['shop_location'],
                'status':b['data']['status'],
                'cancellation_rate':b['data']['cancellation_rate'],
                'item_count':b['data']['item_count'],
                'follower_count':b['data']['follower_count'],
                'response_rate':b['data']['response_rate'],
                'response_time':b['data']['response_time'],
                'preparation_time':b['data']['preparation_time'],
                'following_count':b['data']['account']['following_count'],
                'total_avg_star':b['data']['account']['total_avg_star'],
                'shopurl':'https://shopee.vn/'+ b['data']['account']['username']
               }
        shop_detail_list.append(temp)
        # pause 3-6s before run next page.
        time.sleep(np.random.randint(3,6))
    return shop_detail_list

In [126]:
def get_product_list(shop_detail_list):
    product_list = []
    for s in shop_detail_list:
        temp_url = 'https://shopee.vn/api/v4/search/search_items?by=pop&limit=1&match_id='+str(s['shopid'])+'&newest=0&order=desc&page_type=shop&scenario=PAGE_OTHERS&version=2'
        shop_response = urllib.request.urlopen(temp_url)
        products = json.loads(shop_response.read())

        loop = 0
        total_products = products['total_count']
        if total_products%30 == 0:
            loop = total_products//30
        else:
            loop = total_products//30 + 1
        for index in range(loop):
            temp_url = 'https://shopee.vn/api/v4/search/search_items?by=pop&limit=30&match_id='+str(s['shopid'])+'&newest=' + str(index*50) + '&order=desc&page_type=shop&scenario=PAGE_OTHERS&version=2'
            shop_response = urllib.request.urlopen(temp_url)
            products = json.loads(shop_response.read())
            for i in products['items']:
                product = { 'itemid':i['item_basic']['itemid'],
                            'shopid':i['item_basic']['shopid'],
                           'productname':i['item_basic']['name'],
                           'image':i['item_basic']['image'],
                           'currency':i['item_basic']['currency'],
                           'stock':i['item_basic']['stock'],
                           'status':i['item_basic']['status'],
                           'sold':i['item_basic']['sold'],
                           'historical_sold':i['item_basic']['historical_sold'],
                           'liked_count':i['item_basic']['liked_count'],
                           'view_count':i['item_basic']['view_count'],
                           'catid':i['item_basic']['catid'],
                           'cmt_count':i['item_basic']['cmt_count'],
                           'flag':i['item_basic']['flag'],
                           'cb_option':i['item_basic']['cb_option'],
                           'item_status':i['item_basic']['item_status'], 
                           'price':i['item_basic']['price'],
                           'price_before_discount':i['item_basic']['price_before_discount'],
                           'show_discount':i['item_basic']['show_discount'],
                           'raw_discount':i['item_basic']['raw_discount'],
                           'discount':i['item_basic']['discount'],
                           'tier_variations':i['item_basic']['tier_variations'][0]['options'],
                           'rating_count':i['item_basic']['item_rating']['rating_count'][0],
                           'rating_count_with_context':i['item_basic']['item_rating']['rcount_with_context'],
                           'rating_count_with_image':i['item_basic']['item_rating']['rcount_with_image'],
                           'item_type':i['item_basic']['item_type'],
                           'show_free_shipping':i['item_basic']['show_free_shipping'],
                           'shopee_verified':i['item_basic']['shopee_verified'],
                           'product_url':'https://shopee.vn/'+i['item_basic']['name'].replace(" ","-") + "-i." +  str(i['item_basic']['shopid']) + '.' + str(i['item_basic']['itemid']),
                           'snapshot_date':datetime.date.today()
                        }
                product_list.append(product)
            time.sleep(np.random.randint(3,6))
        time.sleep(np.random.randint(3,6))
    return product_list

In [134]:
def transfer_data_to_db(data,table_name):
    df = pd.DataFrame(data)

    quoted = urllib.parse.quote_plus("DRIVER={ODBC Driver 13 for SQL Server};SERVER=DAVID-HOANG\SQLEXPRESS;DATABASE=shopee;Trusted_connection=yes")
    engine = create_engine('mssql+pyodbc:///?odbc_connect={}'.format(quoted))

    df.to_sql(table_name, schema='dbo', con = engine, method='multi', index=False, if_exists='append')